In [ ]:
#%% 
# Komplett omtag av new_crypto.py
# TODO: Gör om till Binance enligt TDD_krypto
# TODO: Kolla ChatGPT om Binance nycklar och secrets
# TODO: Läs in inflationsdata och gör gemensam df med krypto
# TODO: Inkludera inflationsgraf med grafer ovan

# TODO: Lägg till valet av my own crypto och hantera dem precis som ovan
# TODO: Kopier new_skapa_modeller.ipynb till my_testa_modeller.ipynb
# TODO: Skapa 1 (en) modell för prediction av valfri krypto
# TODO: Testa olika modelltyper i my_skapa_modeller.ipynb

# TODO: Flera sidor?
# TODO: Slutligen: Byt namn till new_crypto.py igen innan publicering
#%%


In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
from pandas.tseries.offsets import DateOffset
from binance.client import Client
import datetime

In [ ]:

@st.cache_data
def get_data():

    api_key = '2jxiCQ8OIWmU4PZH4xfwKEY9KYerDkSWzwNCqoaMzj41eJgWBsSqA3VYqkt2wmdX'
    api_secret = 'YY1Qj1t0JZrE4tQdaBBxT8iwl2tbFalWp1FHjyZ9selBb6OnQ0Oj8aVdiXO7YLMz'

    client = Client(api_key, api_secret)

    # Hämta handelspar
    symbols = client.get_all_tickers()

    # Sätt upp en tom lista för att lagra close-priser
    close_prices = {}

    # Ange den tidsram du vill ha för historiska data
    interval = Client.KLINE_INTERVAL_1DAY
    start_time = "2 month ago UTC"

    # Hämta close-priser för alla kryptovalutor
    dates = None
     #add a counter to print a message every 100 symbols
    counter = 0
    for symbol in symbols:   
        try:
            klines = client.get_historical_klines(symbol['symbol'], interval, start_time)
            if dates is None:
                # Extrahera och konvertera tidsstämplar till datum
                dates = [datetime.datetime.fromtimestamp(int(kline[0]) / 1000).strftime('%Y-%m-%d') for kline in klines]
            close_prices[symbol['symbol']] = [float(kline[4]) for kline in klines]
            # every 100 symbols print a message
            if counter % 100 == 0:
                print(f"Hämtade {counter} symboler")
        except Exception as e:
            print(f"Kunde inte hämta data för {symbol['symbol']}: {e}")
        counter += 1
    # Konvertera close_prices-dikten till en pandas DataFrame
    df = pd.DataFrame.from_dict(close_prices, orient='index').transpose()  
    # Lägg till datum som index för DataFrame
    df.index = pd.to_datetime(dates)                                   # type: ignore
    print(df.head())
    return df
df=get_data()


In [ ]:
# df.head()
# df.to_csv('Tot_crypto.csv')

In [ ]:

st.title('Performance av Kryptovalutor')

months = int(st.number_input('Please enter the number of previous months', min_value=1, max_value=48, value=12))
n_examine = int(st.number_input('Please enter the number of tickers to examine', min_value=1, max_value=25, value=1))

In [ ]:

#%%
def get_returns(df, months):
    # This formula gives the day n months back in time
    #           df.index[-1] - DateOffset(months=months)
    # but we can not be sure it is a trading day so instead:
    old_prices = df[:df.index[-1] - DateOffset(months = months)].tail(1).squeeze()   # n months back in time as a Series
    recent_prices = df.loc[df.index[-1]]
    returns_df = (recent_prices - old_prices) / old_prices
    
    return old_prices.name, returns_df  # date of the old prices and the returns


In [ ]:

#%%
# df = get_data()

In [ ]:
months=1
date, returns_df = get_returns(df, months)

winners, losers = returns_df.nlargest(n_examine), returns_df.nsmallest(n_examine)
winners.name,losers.name = 'Best','Worst'

st.table(winners)
st.table(losers)


In [ ]:

bestPick = st.selectbox('Pick one for graph', winners.index, index=0)
st.line_chart(df[bestPick]) # type: ignore

worstPick = st.selectbox('Pick one for graph', losers.index, index=0)
st.line_chart(df[worstPick]) # type: ignore

